## Create Repository Structure to Download the Data from NOAA API
### *This notebook creates a dictionary of project and weather stations, directories in which weather data will be downloaded*

#### Tasks
1. All folders in data directory.
2. A project dictionary where keys are project names and values are a list of corresponding stations. The stations are in ascending order by distance to project center.
3. A station dictionary where keys are stations names and values are dataframe with corresponding data.

In [5]:
import pandas as pd
import numpy as np
import pickle
import os

In [6]:
all_project_stations = pd.read_csv('./data/all_project_stations.csv')
#print(len(all_project_stations["id"].unique()))
usgs_data = pd.read_csv('./data/uswtdbCSV/uswtdb_v1_1_20180710.csv')
usgs_ge_data = usgs_data[usgs_data["t_manu"] == "GE Wind"]
usgs_ge_data = usgs_ge_data.replace(to_replace=-9999, value=np.nan)
usgs_ge_data["p_name"] = usgs_ge_data["p_name"].str.replace("/","&", regex=False)

# A dictionary with project names as keys and a list of corresponding stations.
# The stations are in ascending order by distance to project center.
project_dict = dict()

# A dictionary with station names as keys and dataframe as values.
station_dict = dict()

# A dictionary with station names as keys and location as values.
station_location_dict = dict()

#### Create directories for each project

In [7]:
projects = usgs_ge_data["p_name"].unique()

for project in projects:
    try:
        os.mkdir("./data/weather/projects/" + project_name)
    except:
        print("Project folder already exists.")

Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project folder already exists.
Project 

#### Creating folders for each station in each project.

In [8]:
for index, row in all_project_stations.iterrows():
    stationid = row['id']
    stationid_dir_name = stationid.replace(":", "_")
    projectid = row["p_name"]
    try:
        os.mkdir("./data/weather/projects/" + projectid + "/" + stationid_dir_name)
    except:
        print("Station folder already exists.")

Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station 

Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station folder already exists.
Station 

#### Creating project dictionary

In [9]:
for project in projects:
    project_dict[project] = []
    project_stations = all_project_stations[all_project_stations["p_name"] == project]
    project_stations_sorted = project_stations.sort_values(by="dist_from_center")
    for index, row in project_stations_sorted.iterrows():
        project_dict[project].append(row["id"])

#### Saving project dictionary to pickle file

In [10]:
pickle_out = open("./data/project_dict.pickle", "wb")
pickle.dump(project_dict, pickle_out)
pickle_out.close()

In [11]:
project_dict["Thunder Ranch"]

['GHCND_USW00013969', 'GHCND_USW00003965', 'GHCND_USC00348501']

#### Creating and saving station_location_dict

In [12]:
for index, row in all_project_stations.iterrows():
    station_location_dict[row["id"]] = row["name"]
    
pickle_out = open("./data/station_location_dict.pickle", "wb")
pickle.dump(station_location_dict, pickle_out)
pickle_out.close()

#### Compiling data from all projects and all stations into single dataframe.

In [13]:
all_project_stations["project_station"] = all_project_stations["p_name"] + all_project_stations["id"]
all_project_stations = all_project_stations.set_index("project_station")
all_project_stations.loc["6th Space Warning SquadronGHCND_USW00054769", "name"]

'PLYMOUTH MUNICIPAL AIRPORT, MA US'

In [14]:
all_weather_df = pd.DataFrame()

In [15]:
directory = "./data/weather/projects/"

for project in project_dict:
    print(project)
    for station in project_dict[project]:
        for filename in os.listdir(directory + project + "/" + station + "/"):
            if filename.endswith(".csv"): 
                #print(directory + project + "/" + station + "/" + filename)
                weather_data = pd.read_csv(directory + project + "/" + station + "/" + filename)
                weather_data["station"] = weather_data["station"].str.replace(":","_", regex=False)
                weather_data["project"] = all_project_stations.loc[project+station, "p_name"]
                weather_data["location"] = all_project_stations.loc[project+station, "name"]
                weather_data["station_lat"] = all_project_stations.loc[project+station, "latitude"]
                weather_data["station_long"] = all_project_stations.loc[project+station, "longitude"]
                weather_data["project_lat_mean"] = all_project_stations.loc[project+station, "query_lat"]
                weather_data["project_long_mean"] = all_project_stations.loc[project+station, "query_long"]
                weather_data["distance_from_center"] = all_project_stations.loc[project+station, "dist_from_center"]
                weather_data["square_diagonal"] = all_project_stations.loc[project+station, "square_diagonal"]
                all_weather_df = all_weather_df.append(weather_data)

print(all_weather_df.info())
all_weather_df.to_csv("./data/all_weather_compiled.csv", index=False)
pickle_out = open("./data/all_weather_df.pickle", "wb")
pickle.dump(all_weather_df, pickle_out)
pickle_out.close()

6th Space Warning Squadron
AFCEE MMR Turbines
AG Land 1
AG Land 2
AG Land 3
AG Land 4
AG Land 5
AG Land 6
ARPA (Prower and Baca County) (Lamar)
Alta I
Alta X
Alta XI
Amazon Wind Farm Texas
Anheuser-Busch Fairfield
Armenia Mountain
Ashtabula II
Ashtabula III
Ashtabula Wind Center
August Wind Energy LLC
Baldwin
Balko Wind
Beech Ridge
Beethoven Wind, LLC
Benson Creek
Benton County Wind Farm
Berkshire Wind
Bethel
Big Horn
Big Timber
Birch Power LLC
Bishop Hill I
Bishop Hill II
Blackstone (Top Crop)
Blackstone (Top Crop) II
Block Island
Blue Canyon V
Blue Summit
Bluff Point
Bobcat Bluff
Brady Wind I
Brady Wind II
Breckinridge
Briscoe
Broken Bow I
Broken Bow II
Brookfield Tehachapi 3
Brookfield Wind Park
Brown County Wind Turbine
Buckeye
Buffalo Dunes
Buffalo Gap, phase II
Bulldog
Burley Butte
Butler Ridge
California Ridge
California State Prison Los Angeles County
Callahan
Camp Grove
Camp Reed
Camp Springs
Camp Springs II (08)
Camp Springs II (4Q07)
Campbell County
Campbell Hill
Canton Moun